In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

In [ ]:
!tar xf spark-3.5.1-bin-hadoop3.tgz

tar: spark-3.5.1-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [1]:
!pip install -q findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=afd55fa1cb464d5954fb9ae1d93876f6a4e6389ada27ae29112c22d58518e9a1
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession, SQLContext, functions, Window # call Sql
from pyspark import SparkContext # call Spark
from google.colab import files # call google to import data file

In [4]:
uplod = files.upload()


Saving netflix_titles.csv to netflix_titles.csv


In [5]:
spark = SparkSession.builder.appName("Gautam").getOrCreate()

In [6]:
spark

In [7]:
df = spark.read.option("header",True).csv("netflix_titles.csv")

In [8]:
df.show(2)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                NULL|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|           NULL|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
+-------+-------+--------------------+---------------+--------------------+-------------+------

Count the number of Movies vs TV Shows

In [ ]:
cnt = df.select("type").where(df.type.isin("TV Show","Movie"))
grp_cnt = cnt.groupby('type').count()
grp_cnt.show()

+-------+-----+
|   type|count|
+-------+-----+
|TV Show| 2676|
|  Movie| 6131|
+-------+-----+



Find the most common rating for movies and TV shows

In [ ]:
mostcom = df.select("type","rating").where(~df.rating.isin("Null"))
mostcom = mostcom.groupby('type','rating').count()
part = Window.partitionBy("type").orderBy(functions.desc('count'))
final = mostcom.withColumn("dense_rank",functions.dense_rank().over(part))
final = final.select("type","rating").where((final.dense_rank==1) & (final.type.isin("Movie","TV Show")))
final.show()

+-------+------+
|   type|rating|
+-------+------+
|  Movie| TV-MA|
|TV Show| TV-MA|
+-------+------+



List all movies released between 2017 to 2021 in a specific year

In [ ]:
movies_17_21 = df.select("title").where((df.release_year >= 2017) & (df.release_year <= 2021))
movies_17_21.show()

+--------------------+
|               title|
+--------------------+
|Dick Johnson Is Dead|
|       Blood & Water|
|           Ganglands|
|Jailbirds New Orl...|
|        Kota Factory|
|       Midnight Mass|
|My Little Pony: A...|
|The Great British...|
|        The Starling|
|Vendetta: Truth, ...|
|    Bangkok Breaking|
|        Je Suis Karl|
|Confessions of an...|
|Crime Stories: In...|
|   Dear White People|
|Europe's Most Dan...|
|     Falsa identidad|
|           Intrusion|
|              Jaguar|
|Monsters Inside: ...|
+--------------------+
only showing top 20 rows



count the number of movies and TV shows released each year from 2011 onward?

In [ ]:
list_11 = df.select("title","release_year").where(df.release_year >= 2011)
list_11 = list_11.groupby('release_year').count().orderBy('release_year')
list_11.show()

+------------+-----+
|release_year|count|
+------------+-----+
|        2011|  185|
|        2012|  237|
|        2013|  288|
|        2014|  352|
|        2015|  559|
|        2016|  901|
|        2017| 1030|
|        2018| 1145|
|        2019| 1026|
|        2020|  952|
|        2021|  589|
+------------+-----+



Find the top 5 countries with the most content on Netflix

In [ ]:
seprate_country = df.select("*",functions.explode(functions.split(df.country,",\s*")).alias("countrys"))
seprate_country=seprate_country.select("countrys")
cnt_seprate_country=seprate_country.groupby('countrys').count()
partition = Window.orderBy(functions.desc('count'))
rank = cnt_seprate_country.withColumn("dense_rank",functions.dense_rank().over(partition))
rank = rank.select("countrys").where(rank.dense_rank <= 5)
rank.show()

+--------------+
|      countrys|
+--------------+
| United States|
|         India|
|United Kingdom|
|        Canada|
|        France|
+--------------+



Identify the longest movie

In [ ]:
lng_movie = df.select(
    "type",
    "title",
    functions.regexp_replace("duration", " min", "").cast("int").alias("dura")
).where(
    (df.type == 'Movie') & (functions.col("duration").isNotNull())
).filter(
    functions.col("dura").isNotNull()
).orderBy(functions.col("dura").desc())

lng_movie = lng_movie.where(lng_movie.dura <= 1000)   #longest movies till now is 873 min

lng_movie.show(1)

+-----+--------------------+----+
| type|               title|dura|
+-----+--------------------+----+
|Movie|Black Mirror: Ban...| 312|
+-----+--------------------+----+
only showing top 1 row



Identifying the Directors with the Most Movie Productions on Netflix

In [ ]:
directors = df.select("*",functions.explode(functions.split(df.director,",\s")).alias('dire'))
directors = directors.select("*", functions.trim(functions.col("dire")).alias("direc"))
directors = directors.select(
    "*",
    functions.trim(functions.col("dire")).alias("trimmed_direc")
).withColumn(
    "direc",
    functions.col("trimmed_direc")
).drop("trimmed_direc")

grp_dirct = directors.groupby('direc').count().orderBy(functions.col('count').desc())
# grp_dirct.show(grp_dirct.count())
grp_dirct.show(1)

+-------------+-----+
|        direc|count|
+-------------+-----+
|Rajiv Chilaka|   22|
+-------------+-----+
only showing top 1 row



List all the movies by director 'Rajiv Chilaka'

In [ ]:
Rajiv_Chilaka = df.select("*",functions.explode(functions.split(df.director,",\s")).alias("dire"))
Rajiv_Chilaka = Rajiv_Chilaka.select("*").where(Rajiv_Chilaka.dire == "Rajiv Chilaka")
Rajiv_Chilaka.show(Rajiv_Chilaka.count())

+-------+-----+--------------------+--------------------+--------------------+-------+---------------+------------+------+--------+--------------------+--------------------+-------------+
|show_id| type|               title|            director|                cast|country|     date_added|release_year|rating|duration|           listed_in|         description|         dire|
+-------+-----+--------------------+--------------------+--------------------+-------+---------------+------------+------+--------+--------------------+--------------------+-------------+
|   s407|Movie|Chhota Bheem - Ne...|       Rajiv Chilaka|Vatsal Dubey, Jul...|   NULL|  July 22, 2021|        2013| TV-Y7|  64 min|Children & Family...|Things get spooky...|Rajiv Chilaka|
|   s408|Movie|Chhota Bheem & Ga...|       Rajiv Chilaka|Vatsal Dubey, Jul...|   NULL|  July 22, 2021|        2009| TV-Y7|  68 min|Children & Family...|Bheem and Lord Ga...|Rajiv Chilaka|
|   s409|Movie|Chhota Bheem & Kr...|       Rajiv Chilaka|Vat

List all TV shows with more than 5 seasons

In [ ]:
tv_show = df.select("type","title","duration",
                    functions.regexp_replace(functions.regexp_replace("duration"," Seasons","")," Season","" ).cast("int").alias("dura"))\
.where(df.type == "TV Show").orderBy(functions.col("dura").desc())
tv_show = tv_show.select("type","title","dura").where(tv_show.dura > 5)
tv_show.show()

+-------+--------------------+----+
|   type|               title|dura|
+-------+--------------------+----+
|TV Show|      Grey's Anatomy|  17|
|TV Show|        Supernatural|  15|
|TV Show|                NCIS|  15|
|TV Show|COMEDIANS of the ...|  13|
|TV Show|           Heartland|  13|
|TV Show|        Red vs. Blue|  13|
|TV Show|   Trailer Park Boys|  12|
|TV Show|      Criminal Minds|  12|
|TV Show|              Cheers|  11|
|TV Show|             Frasier|  11|
|TV Show|    The Walking Dead|  10|
|TV Show|    Shameless (U.S.)|  10|
|TV Show|       Stargate SG-1|  10|
|TV Show|Danger Mouse: Cla...|  10|
|TV Show|          Dad's Army|  10|
|TV Show|LEGO Ninjago: Mas...|  10|
|TV Show|             Friends|  10|
|TV Show|The Great British...|   9|
|TV Show|    Call the Midwife|   9|
|TV Show|          Still Game|   9|
+-------+--------------------+----+
only showing top 20 rows



Count the number of content items in each genre

In [ ]:
genre = df.select("show_id","type","listed_in",functions.explode(functions.split(df.listed_in,",\s")).alias("genre"))
genre = genre.groupby('genre').count().orderBy(functions.col('count').desc())
genre.show()

+--------------------+-----+
|               genre|count|
+--------------------+-----+
|International Movies| 2748|
|              Dramas| 2419|
|            Comedies| 1670|
|International TV ...| 1350|
|       Documentaries|  866|
|  Action & Adventure|  857|
|           TV Dramas|  762|
|  Independent Movies|  751|
|Children & Family...|  641|
|     Romantic Movies|  616|
|         TV Comedies|  580|
|           Thrillers|  573|
|      Crime TV Shows|  469|
|            Kids' TV|  451|
|          Docuseries|  395|
|    Music & Musicals|  373|
|   Romantic TV Shows|  370|
|       Horror Movies|  357|
|     Stand-Up Comedy|  343|
|          Reality TV|  254|
+--------------------+-----+
only showing top 20 rows



Find each year and the total numbers of content release in India on netflix. return top 5 year with highest avg content release!

In [17]:
india_df = df.select("*", functions.explode(functions.split(df.country,",\s")).alias("countries"))
india_df = india_df.select("title","release_year","countries").where(india_df.countries == "India")
india_df = india_df.groupby('release_year').count().orderBy(functions.col('count').desc())
windo = Window.orderBy(functions.col('count').desc())
dense_rk = india_df.withColumn("dense_rank",functions.dense_rank().over(windo))
final = dense_rk.where(dense_rk.dense_rank <= 5)
# dense_rk.show()
final.show()

+------------+-----+----------+
|release_year|count|dense_rank|
+------------+-----+----------+
|        2017|  111|         1|
|        2018|  101|         2|
|        2019|   93|         3|
|        2016|   80|         4|
|        2020|   77|         5|
+------------+-----+----------+



List all movies that are documentaries

In [22]:
document = df.select("*",functions.explode(functions.split(df.listed_in,",\s")).alias("genric"))
document = document.select("type","title","genric").where((document.genric == "Documentaries") & (document.type == "Movie"))
document.show()

+-----+--------------------+-------------+
| type|               title|       genric|
+-----+--------------------+-------------+
|Movie|Dick Johnson Is Dead|Documentaries|
|Movie|Europe's Most Dan...|Documentaries|
|Movie|My Heroes Were Co...|Documentaries|
|Movie|          Schumacher|Documentaries|
|Movie|Blood Brothers: M...|Documentaries|
|Movie|The Women and the...|Documentaries|
|Movie|If I Leave Here T...|Documentaries|
|Movie|Untold: Breaking ...|Documentaries|
|Movie|       Final Account|Documentaries|
|Movie|Snervous Tyler Oa...|Documentaries|
|Movie|Untold: Crime & P...|Documentaries|
|Movie|Bob Ross: Happy A...|Documentaries|
|Movie|    The River Runner|Documentaries|
|Movie|         Count Me In|Documentaries|
|Movie|Untold: Caitlyn J...|Documentaries|
|Movie|Memories of a Mur...|Documentaries|
|Movie|Untold: Deal With...|Documentaries|
|Movie|Misha and the Wolves|Documentaries|
|Movie|Untold: Malice at...|Documentaries|
|Movie|City of God: 10 Y...|Documentaries|
+-----+----

List all content without a director

In [27]:
director = df.select("*").where(df.director.isNull())
director.show()

+-------+-------+--------------------+--------+--------------------+--------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|director|                cast|       country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------+--------------------+--------------+------------------+------------+------+---------+--------------------+--------------------+
|     s2|TV Show|       Blood & Water|    NULL|Ama Qamata, Khosi...|  South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s4|TV Show|Jailbirds New Orl...|    NULL|                NULL|          NULL|September 24, 2021|        2021| TV-MA| 1 Season|Docuseries, Reali...|Feuds, flirtation...|
|     s5|TV Show|        Kota Factory|    NULL|Mayur More, Jiten...|         India|September 24, 2021|        2021| TV-MA|2 S

Thank You